# Important libraries

In [17]:
import win32com.client
import sys
import time
import os
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Openserver Connection

Openserver class with methods to connect to the server, send commands, and disconnect from the server.

In [18]:
class OpenServer():
    "Class for holding ActiveX reference. Allows license disconnection"
    def __init__(self):
        self.status = "Disconnected"
        self.OSReference = None
    
    def Connect(self):
        self.OSReference = win32com.client.Dispatch("PX32.OpenServer.1")
        self.status = "Connected"
        print("OpenServer connected")
        
    def Disconnect(self):
        self.OSReference = None
        self.status = "Disconnected"
        print("OpenServer disconnected")

In [19]:
def GetAppName(sv):
    # function for returning app name from tag string
    pos = sv.find(".")
    if pos < 2:
        sys.exit("GetAppName: Badly formed tag string")
    app_name = sv[:pos]
    if app_name.lower() not in ["prosper", "mbal", "gap", "pvt", "resolve",
                                   "reveal"]:
        sys.exit("GetAppName: Unrecognised application name in tag string")
    return app_name


def DoCmd(OpenServe, cmd):
    # perform a command and check for errors
    lerr = OpenServe.OSReference.DoCommand(cmd)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoCmd: " + err)


def DoSet(OpenServe, sv, val):
    # set a value and check for errors
    lerr = OpenServe.OSReference.SetValue(sv, val)
    app_name = GetAppName(sv)
    lerr = OpenServe.OSReference.GetLastError(app_name)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoSet: " + err)
    
def DoGet(OpenServe, gv):
    # get a value and check for errors
    get_value = OpenServe.OSReference.GetValue(gv)
    app_name = GetAppName(gv)
    lerr = OpenServe.OSReference.GetLastError(app_name)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoGet: " + err)
    return get_value


def DoSlowCmd(OpenServe, cmd):
    # perform a command then wait for command to exit and check for errors
    step = 0.001
    app_name = GetAppName(cmd)
    lerr = OpenServe.OSReference.DoCommandAsync(cmd)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoSlowCmd: " + err)
    while OpenServe.OSReference.IsBusy(app_name) > 0:
        if step < 2:
            step = step*2
        time.sleep(step)
    lerr = OpenServe.OSReference.GetLastError(app_name)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoSlowCmd: " + err)


def DoGAPFunc(OpenServe, gv):
    DoSlowCmd(gv)
    DoGAPFunc = DoGet(OpenServe, "GAP.LASTCMDRET")
    lerr = OpenServe.OSReference.GetLastError("GAP")
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("DoGAPFunc: " + err)
    return DoGAPFunc


def OSOpenFile(OpenServe, theModel, appname):
    DoSlowCmd(OpenServe, appname + '.OPENFILE ("' + theModel + '")')
    lerr = OpenServe.OSReference.GetLastError(appname)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("OSOpenFile: " + err)


def OSSaveFile(OpenServe, theModel, appname):
    DoSlowCmd(OpenServe, appname + '.SAVEFILE ("' + theModel + '")')
    lerr = OpenServe.OSReference.GetLastError(appname)
    if lerr > 0:
        err = OpenServe.OSReference.GetErrorDescription(lerr)
        OpenServe.Disconnect()
        sys.exit("OSSaveFile: " + err)

# Single Well calculation

In [20]:
#gas availability and increment steps
gas_available = float(input('Please enter available gas'))
increment = float(input('Please enter the increment value'))
gas_injection_array = np.arange(0, gas_available, increment)
oil_rate_array = []

In [21]:
# Initialises an 'OpenServer' class

petex = OpenServer()

# Creates ActiveX reference and holds a license

petex.Connect()

# Perform functions

cwd = os.getcwd()
OSOpenFile(petex, cwd + r'\well_1.Out', 'PROSPER')
for i in range(0, len(gas_injection_array)):
    command = f'PROSPER.ANL.SYS.Sens.SensDB.Sens[138].Vals[{i}]'
    DoSet(petex, command,\
        gas_injection_array[i])
DoCmd(petex, 'PROSPER.ANL.SYS.CALC')
for i in range(0, len(gas_injection_array)):
    value = f'PROSPER.OUT.SYS.Results[{i}].Sol.OilRate'
    oil_rate_array.append(np.round(float((DoGet(petex, value ))),2))
OSSaveFile(petex, cwd + r'\well_1.Out', 'PROSPER')


#creating dataframe for gas injection rate and oil rate values
df = pd.DataFrame({'oil_rate':oil_rate_array, 'gas_injection':gas_injection_array})
df.head()

OpenServer connected


,oil_rate,gas_injection
0,1749.18,0.0
1,1781.27,5.0


In [22]:
fig = px.line(df, x='gas_injection', y='oil_rate', markers=True)
fig.update_layout(title='Sensitivity Plot',
                   xaxis_title='Gaslift Gas Injection Rate(MMscf/day)',
                   yaxis_title='Oil Rate(STB/day)',
                   title_x=0.5, 
                   xaxis={
                       'range':[0, df['gas_injection'].max()+0.2]
                   })
fig.update_xaxes(nticks=50)
fig.update_yaxes(nticks=20)
fig.show()

# Multiple well calculation

In [23]:
#gas availability and increment steps
number_of_wells = int(input('Please enter the number of wells:'))
gas_available = float(input('Please enter available gas'))
increment = float(input('Please enter the increment value'))
gas_injection_array = np.arange(0, gas_available, increment)
df=pd.DataFrame()
df['gas_injection_array'] = gas_injection_array

In [24]:
# Initialises an 'OpenServer' class

petex = OpenServer()

# Creates ActiveX reference and holds a license

petex.Connect()
oil_rates_for_wells = {}

# Perform functions

cwd = os.getcwd()
for j in range(1,number_of_wells+1):
    #opening well file
    OSOpenFile(petex, cwd + f'\well_{j}.Out', 'PROSPER')
    print(f'Well {j} opened')
    #oil rates calculation
    for i in range(0,len(gas_injection_array)):
        command = f'PROSPER.ANL.SYS.Sens.SensDB.Sens[138].Vals[{i}]'
        DoSet(petex, command,gas_injection_array[i])
    DoCmd(petex, 'PROSPER.ANL.SYS.CALC')
    oil_rates = [] #list of oil rates for 1 well
    for i in range(0, len(gas_injection_array)):
        value = f'PROSPER.OUT.SYS.Results[{i}].Sol.OilRate'
        oil_rates.append(np.round(float(DoGet(petex, value)),2))
    oil_rates_for_wells[f'well_{j}'] = oil_rates
    #closing file   
    OSSaveFile(petex, cwd + f'\well_{j}.Out', 'PROSPER')
    print(f'Well {j} closed')

#merging to main dataframe
for i in range(1,number_of_wells+1):
    df[f'Well_{i}'] = oil_rates_for_wells[f'well_{i}']

df.head()

OpenServer connected
Well 1 opened
Well 1 closed
Well 2 opened
Well 2 closed
Well 3 opened
Well 3 closed
Well 4 opened
Well 4 closed
Well 5 opened
Well 5 closed
Well 6 opened
Well 6 closed
Well 7 opened
Well 7 closed
Well 8 opened
Well 8 closed
Well 9 opened
Well 9 closed
Well 10 opened
Well 10 closed


,gas_injection_array,Well_1,Well_2,Well_3,Well_4,Well_5,Well_6,Well_7,Well_8,Well_9,Well_10
0,0.0,1749.18,1657.35,1576.44,1362.42,997.05,1800.94,2071.36,1205.72,2003.49,1413.76
1,0.2,1767.97,1677.71,1597.90,1387.41,1027.60,1820.36,2085.03,1233.84,2018.35,1437.59
2,0.4,1781.27,1692.24,1613.28,1405.79,1051.22,1832.75,2093.80,1254.66,2026.18,1455.11
3,0.6,1790.62,1702.57,1623.94,1419.53,1067.58,1841.38,2099.33,1269.66,2031.11,1468.18
4,0.8,1796.92,1710.09,1633.80,1431.86,1079.76,1847.08,2101.67,1280.88,2033.69,1477.87


In [28]:
fig = go.Figure()
for i in range(1,number_of_wells+1):
    
    fig.add_trace(go.Scatter(x=df['gas_injection_array'], y=df[f'Well_{i}'],\
                            mode='lines', name=f'Well_{i}'))
fig.update_layout(title='Sensitivity Plot',
                   xaxis_title='Gaslift Gas Injection Rate(MMscf/day)',
                   yaxis_title='Oil Rate(STB/day)',
                   title_x=0.5, 
                   xaxis={
                       'range':[0, df['gas_injection_array'].max()+0.2]
                   })
fig.update_xaxes(nticks=50)
fig.update_yaxes(nticks=20)
fig.show()

# Optimization of gaslift injection rate